In [2]:
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

In [3]:
## SQL query to extract additonal costs of NHS price concessions. 
## Modfied version of one on site i.e. done at a "organisational level" 
q='''SELECT
  ncso.date AS month,
  product.bnf_code AS bnf_code,
  product.name AS product_name,
  rx.quantity AS quantity,
  rx.pct AS pct,
  ccg.org_type,
  ccg.name,
  dt.price_pence
    * rx.quantity
    * CASE WHEN
        -- For some presentations "quantity" means "number of packs" rather
        -- than e.g. tablets. In these cases we don't want to divide by the
        -- quantity value of a pack. This is implemented via a flag in our
        -- databse but this data isn't in BiqQuery so we just have a hardcoded
        -- list of BNF codes here
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    -- This is the "discount factor" which applies the National Average 7.2%
    -- discount to estimate Actual Cost from Net Ingredient Cost and also
    -- converts figures from pence to pounds
    * 0.00928
    AS tariff_cost,
  COALESCE(ncso.price_concession_pence - dt.price_pence, 0)
    * rx.quantity
    * CASE WHEN
        product.bnf_code in ('0206010F0AACJCJ')
      THEN
        1
      ELSE
        1 / vmpp.qtyval
      END
    * 0.00928
    AS additional_cost,
  ncso.date != rx.month AS is_projection
FROM
  dmd.ncsoconcession AS ncso
JOIN
  dmd.tariffprice AS dt
ON
  ncso.vmpp = dt.vmpp AND ncso.date = dt.date
JOIN
  dmd.product AS product
ON
  dt.product=product.dmdid
JOIN
  dmd.vmpp AS vmpp
ON
  vmpp.vppid=ncso.vmpp
JOIN
  hscic.presentation AS presentation
ON
  presentation.bnf_code = product.bnf_code
JOIN
  hscic.prescribing AS rx
ON
  rx.bnf_code = product.bnf_code
JOIN
  hscic.ccgs AS ccg #this joins to our CCG organisational data
ON
  rx.pct = ccg.code
AND
-- Where we have prescribing data for the corresponding month we use
-- that, otherwise we use the latest month of prescribing data to
-- produce an estimate
(
  rx.month = ncso.date
  OR
  (
    -- This should be set to the latest date for which we have prescribing
    -- data
    rx.month = TIMESTAMP('2018-12-01')
    AND
    ncso.date > rx.month
  )
)'''
    
all_ncso = pd.read_gbq(q, GBQ_PROJECT_ID, verbose = False, dialect = 'standard')

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas_gbq\gbq.py:556: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  "verbosity", FutureWarning, stacklevel=1)


In [4]:
## Selecting nonCCG organisations to work 
nonccg_ncso  = all_ncso.loc[all_ncso.org_type !='CCG']
nonccg_ncso.head()

,month,bnf_code,product_name,quantity,pct,org_type,name,tariff_cost,additional_cost,is_projection
22,2014-09-01,1001010P0AAAIAI,Naproxen 500mg gastro-resistant tablets,28,RY5,Unknown,None,3.660960,0.310880,False
1364,2018-10-01,1001010P0AAADAD,Naproxen 250mg tablets,434,NTP,Unknown,None,13.233280,19.849920,False
1719,2018-11-01,0409010N0AAABAB,Co-careldopa 25mg/100mg tablets,112,RYW,Unknown,None,6.589542,3.804058,False
1739,2018-11-01,0202020L0AABDBD,Furosemide 40mg tablets,112,RY8,Unknown,None,2.041600,8.797440,False
1806,2018-11-01,040201030AAABAB,Risperidone 2mg tablets,112,RLY,Unknown,None,2.373205,17.322667,False


### 2018 Impact - all months, all orgs, all meds

In [5]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [6]:
## Restricting data to 2018 calendar year
nonccg_ncso_thisyear = nonccg_ncso.loc[(nonccg_ncso["month"]>="2018-01-01") & (nonccg_ncso["month"]<="2018-12-01")]
nonccg_ncso_thisyear.head()

,month,bnf_code,product_name,quantity,pct,org_type,name,tariff_cost,additional_cost,is_projection
1364,2018-10-01,1001010P0AAADAD,Naproxen 250mg tablets,434,NTP,Unknown,None,13.23,19.85,False
1719,2018-11-01,0409010N0AAABAB,Co-careldopa 25mg/100mg tablets,112,RYW,Unknown,None,6.59,3.80,False
1739,2018-11-01,0202020L0AABDBD,Furosemide 40mg tablets,112,RY8,Unknown,None,2.04,8.80,False
1806,2018-11-01,040201030AAABAB,Risperidone 2mg tablets,112,RLY,Unknown,None,2.37,17.32,False
1977,2018-11-01,0103010T0AAAAAA,Ranitidine 150mg tablets,120,AD9,Unknown,None,1.87,0.54,False


In [8]:
## Total cost impact of price consessions this year to nonCCG organisations. This will return an "unknown name"
nonccg_ncso_thisyear.groupby('org_type').sum().sort_values(by = 'additional_cost', ascending = False)

,quantity,tariff_cost,additional_cost,is_projection
org_type,,,,
Unknown,9250711,1276379.41,5627890.92,False


In [11]:
## nonccg organisations grouped by indivual preparations
nonccg_ncso_thisyear.groupby('product_name').sum().sort_values(by = 'additional_cost', ascending = False)

,quantity,tariff_cost,additional_cost,is_projection
product_name,,,,
Buprenorphine 8mg sublingual tablets sugar free,2662817,682062.04,3646366.48,False
Buprenorphine 2mg sublingual tablets sugar free,3450506,414783.01,1823930.78,False
Phenoxymethylpenicillin 125mg/5ml oral solution,561600,17613.44,18634.74,False
Phenoxymethylpenicillin 250mg/5ml oral solution,331900,13230.31,9179.18,False
Naproxen 500mg tablets,48466,1989.28,7665.94,False
Diamorphine 30mg powder for solution for injection ampoules,7637,16243.72,7172.18,False
Metronidazole 400mg tablets,43417,4715.31,6723.62,False
Risperidone 500microgram tablets,69363,1931.19,6346.26,False
Diamorphine 10mg powder for solution for injection ampoules,7622,16571.03,6285.61,False


In [12]:
## monthly impact of price concessions for nonCCG organisations
nonccg_ncso_thisyear.groupby('month').sum()

,quantity,tariff_cost,additional_cost,is_projection
month,,,,
2018-01-01,222752,14003.23,12341.81,False
2018-02-01,527126,21353.33,20029.98,False
2018-03-01,654052,24725.51,23698.88,False
2018-04-01,988802,126949.47,44418.85,False
2018-05-01,972980,133551.95,870681.17,False
2018-06-01,795898,122896.73,564727.31,False
2018-07-01,830210,121639.71,626966.71,False
2018-08-01,820650,140137.93,480764.05,False
2018-09-01,764163,131730.37,738909.13,False


### Impact on individual organisations

In [15]:
## assess impact on each organisation "PCT"
nonccg_ncso_orgs = nonccg_ncso_thisyear.groupby('pct').sum().sort_values(by = 'additional_cost', ascending = False)

In [16]:
## import councils from NHS Digital ODS codes
df_councils = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\nonCCG Concessions\Lauth.csv')

In [17]:
## import "independent providers" from NHS Digital ODS codes
df_indprov = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\nonCCG Concessions\ephp.csv')
df_councils['pct'] = df_councils['pct'].astype('str')

In [18]:
## import hospitals from NHS Digital ODS codes
df_hosp = pd.read_csv(r'C:\Users\bmackenna\Documents\GitHub\nonCCG Concessions\etr.csv')

In [19]:
importmerge_orgnames_part1 = nonccg_ncso_orgs.reset_index().merge(df_councils[['pct','name']],  how = "outer", on = 'pct')

In [20]:
importmerge_orgnames_part2 = importmerge_orgnames_part1.merge(df_indprov[['pct','name']],  how="outer", on='pct', suffixes=("_council","_independent_provider"))

In [21]:
fullmerge = importmerge_orgnames_part2.merge(df_hosp[['pct','name']],  how="outer", on='pct')

In [22]:
fullmerge['purchaser_name'] = fullmerge['name_council'].fillna('') + fullmerge['name_independent_provider'].fillna('') + fullmerge['name'].fillna('')

In [23]:
purchaser_list = fullmerge.drop('name_council', axis=1) ####getting multiple errors when trying to drop all at once

In [24]:
purchaser_list1 = purchaser_list.drop('name', axis=1)

In [26]:
purchaser_list2 = purchaser_list1.drop('name_independent_provider', axis=1)
purchaser_list2[['purchaser_name', 'pct', 'additional_cost']]

,purchaser_name,pct,additional_cost
0,"CHANGE, GROW, LIVE",NMS,2091871.32
1,TURNING POINT,NKI,582024.50
2,ADDACTION,NI3,226144.28
3,LEEDS CITY COUNCIL,212,181742.50
4,"NORTHUMBERLAND, TYNE AND WEAR NHS FOUNDATION T...",RX4,174186.77
5,MIDDLESBROUGH COUNCIL,112,164438.04
6,NEWCASTLE-UPON-TYNE CITY COUNCIL,107,124219.57
7,SPECTRUM COMMUNITY HEALTH - CIC,NL1,122254.48
8,THE FORWARD TRUST,DG3,116849.06
9,STOCKTON-ON-TEES BOROUGH COUNCIL,114,113859.05
